# Policy gradient actually 

In [2]:
import numpy as np
import gym
import multiprocessing
# from stable_baselines.common.vec_env import SubprocVecEnv
# from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv

In [ ]:
'''
batch = [
    [state1, action1, reward_to_go1],
    [state2, action2, reward_to_go2] # reward_to_go2 < reward_to_go1
]

log_probs = reducesum(encoded_actions * log_softmax(logits), axis=1)
loss = -reduce_mean(rewards * log_probs)
optimizer.minimize(loss)

to desync envs: make some random steps in each 


'''

In [3]:
ENV_NAME = 'CartPole-v0'
NPROC = multiprocessing.cpu_count()
BATCH_SIZE = 256

In [4]:
def make_env(name, seed):
    def f():
        env = gym.make(name)
        env.seed(seed)
        return env
    return f
envs = SubprocVecEnv([make_env(ENV_NAME, i) for i in range(NPROC)])
print(envs)

In [ ]:
class PGAgent(object):
    
    def __init__(
        self, envs, batch_size
    ):
        self.observation_size = 0
        self.actions_size = 0
        self._policy = tf.nn(input=(None, OBSERVATIONS_SIZE), (None, ACTIONS_SIEZ))
        
    
    def get_batch(self):
        while True:
            yield np.empty([0, 3])
            
    def policy(self, observations):
        return self._policy
    
    def train(self, n_epochs, lr):
        for i in range n_epochs:
            batch = next(self.get_batch)
            self.__train_on_batch(batch)
            
    def __train_on_batch(self, batch):
            #  here actually do something
            pass

In [ ]:
if __name__ == '__main__':
    #train and evel
    pass